# Langchain - Cadenas de extracción

En este post vamos a ver cómo usar las [tool-callings](https://python.langchain.com/docs/concepts/tool_calling) de los [chat models](https://python.langchain.com/docs/concepts/chat_models) para extraer información estructurada de textos no estructurados

Además vamos a ver cómo [Few-shot prompting](https://python.langchain.com/docs/concepts/few_shot_prompting) ayuda a mejorar el rendimiento en este contexto

## Schema

Primero, necesitamos describir qué información queremos extraer del texto.

Usaremos ``Pydantic`` para definir un esquema de ejemplo para extraer información personal.

In [1]:
from typing import Optional
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

Hay dos buenas prácticas al definir el esquema:

 * Documentar los atributos y el esquema: Esta información se envía al LLM y se utiliza para mejorar la calidad de la extracción de información.
 * No forzar al LLM a inventar información: Hemos usamos ``Optional`` para los atributos que permiten que el LLM devuelva ``None`` si no sabe la respuesta.

 > **Importante**: Para un mejor rendimiento, documentar bien el esquema y asegurarse de que el modelo no esté obligado a devolver los resultados si no hay información que se extraerá en el texto

## Extractor

Creamos un extractor de información utilizando el esquema que hemo definido anteriormente

In [2]:
from typing import Optional
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

Aquí hemos usado [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html) y [MessagesPlaceholder](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.MessagesPlaceholder.html)

Ahora necesitamos usar un modelo que admita la herramieta de llamadas de función, podemos encontrar un modelo que lo soporte en [Featured Providers](https://python.langchain.com/docs/integrations/chat/#featured-providers)

En nuestro caso vamos a usar los modelos de OpenAI

## Creamos un LLM

In [3]:
from langchain.chat_models import init_chat_model
import os
import dotenv

dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = init_chat_model("gpt-4o-mini", model_provider="openai", api_key=OPENAI_API_KEY)

## LLM estructurado

Ahora podemos crear un LLM que genere salidas estructuradas de acuerdo al esquema ``Person`` que hemos definido anteriormente.

In [4]:
structured_llm = llm.with_structured_output(schema=Person)

Ya podemos generar una salida estructurada a partir de un texto no estructurado

In [5]:
text = "Alan Smith is 6 feet tall and has blond hair."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


Person(name='Alan Smith', hair_color='blond', height_in_meters='1.83')

## Múltiples Entidades

En la mayoría de los casos, debe extraer una lista de entidades en lugar de una sola entidad.

Esto se puede lograr fácilmente usando ``pydantic`` anidando modelos uno dentro del otro

In [6]:
from typing import List, Optional
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )


class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

 > **Importante**: Los resultados de extracción pueden no ser perfectos aquí. Usar [Ejemplos de Referencia](https://python.langchain.com/docs/how_to/#extraction) puede mejorar la calidad de la extracción

Creamos un llm estructurado con los esquemas anteriores

In [7]:
structured_llm = llm.with_structured_output(schema=Data)

Ahora podemos obtener múltiples datos estructurados a partir de un texto no estructurado

In [8]:
text = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTP

Data(people=[Person(name='Jeff', hair_color='black', height_in_meters='1.83'), Person(name='Anna', hair_color='black', height_in_meters=None)])

## Mejora de resultados con ejemplos de referencia

El comportamiento de las aplicaciones LLM se puede dirigir usando unos [pocos ejemplos](https://python.langchain.com/docs/concepts/few_shot_prompting). Para modelos de chat esto puede tomar la forma de una secuencia de los ejemplos y generar una respuesta que siga los ejemplos dados.

Las [salidas estructuradas](https://python.langchain.com/docs/concepts/structured_outputs/) a menudo utilizan [tool-callings](https://python.langchain.com/docs/concepts/tool_calling) por debajo. Esto implica típicamente la generación de [AIMessage](https://python.langchain.com/docs/concepts/messages/#aimessage)s que contienen `tool-callings`, o [tool messages](https://python.langchain.com/docs/concepts/messages/#toolmessage) conteniendo los resultados de las [tool-callings]. ¿Cómo debería ser una secuencia de mensajes en este caso?

Diferente [proveedores de modelos de chat](https://python.langchain.com/docs/integrations/chat/) imponen diferentes requisitos para secuencias de mensajes válidas.

Algunos aceptan una secuencia de mensajes de la forma:

 * Mensaje de usuario
 * Mensaje de IA con llamada de herramienta
 * Mensaje de herramienta con resultado

Otros requieren un mensaje final de IA que contenga algún tipo de respuesta.

LangChain incluye la función [tool_example_to_messages](https://python.langchain.com/api_reference/core/utils/langchain_core.utils.function_calling.tool_example_to_messages.html) eso generará una secuencia válida para la mayoría de los proveedores de modelos.

Simplifica la generación de ejemplos estructurados de pocos ejemplos simplemente requiriendo representaciones pydantic de las llamadas de herramientas correspondientes.

Vamos a probarlo con un ejemplo.

Podemos convertir pares de cadenas de entrada y los objetos de Pydantic para esas entradas en una secuencia de mensajes que se pueden proporcionar a un modelo de chat como ejemplo.

Por debajo, LangChain formateará las ``tool-callings`` al formato requerido de cada proveedor.

In [10]:
examples = [
    (
        "The ocean is vast and blue. It's more than 20,000 feet deep.",
        Data(people=[]),
    ),
    (
        "Fiona traveled far from France to Spain.",
        Data(people=[Person(name="Fiona", height_in_meters=None, hair_color=None)]),
    ),
]

In [11]:
from langchain_core.utils.function_calling import tool_example_to_messages

messages = []

for txt, tool_call in examples:
    if tool_call.people:
        # This final message is optional for some providers
        ai_response = "Detected people."
    else:
        ai_response = "Detected no people."
    messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))

/var/folders/8j/g2d036fx07l30pfh53c8kb180000gn/T/ipykernel_1468/3178671663.py:11: LangChainBetaWarning: The function `tool_example_to_messages` is in beta. It is actively being worked on, so the API may change.
  messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))


Al inspeccionar el resultado, vemos que estos dos pares de ejemplos generaron ocho mensajes

In [12]:
for message in messages:
    message.pretty_print()

================================ Human Message =================================

The ocean is vast and blue. It's more than 20,000 feet deep.
================================== Ai Message ==================================
Tool Calls:
  Data (b32fc767-4ee7-445f-9b91-8be1e1c35400)
 Call ID: b32fc767-4ee7-445f-9b91-8be1e1c35400
  Args:
    people: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

Detected no people.
================================ Human Message =================================

Fiona traveled far from France to Spain.
================================== Ai Message ==================================
Tool Calls:
  Data (e01a030d-0710-49b5-982e-f8c0c7c934ae)
 Call ID: e01a030d-0710-49b5-982e-f8c0c7c934ae
  Args:
    people: [{'name': 'Fiona', 'hair_color': None, 'height_in_meters': None}]
================================

Como se puede ver para el texto ``The ocean is vast and blue. It's more than 20,000 feet deep``se han generado los siguientes mensajes:

 * **Ai Message**
 ```
 Tool Calls:
  Data (b32fc767-4ee7-445f-9b91-8be1e1c35400)
 Call ID: b32fc767-4ee7-445f-9b91-8be1e1c35400
  Args:
    people: []
 ```
 * **Tool Message**
 ```
 You have correctly called this tool
 ```
 * **Ai Message**
 ```
 Detected no people.
 ```

Comparemos el rendimiento con y sin estos mensajes. Por ejemplo, pasemos un mensaje para el que pretendemos que no se extraiga a ninguna gente

In [13]:
message_no_extraction = {
    "role": "user",
    "content": "The solar system is large, but earth has only 1 moon.",
}

structured_llm = llm.with_structured_output(schema=Data)
structured_llm.invoke([message_no_extraction])

Data(people=[Person(name='Earth', hair_color='None', height_in_meters='0.00')])

En este ejemplo, el modelo ha generado erróneamente registros de personas

Ahora probamos pasándole unos pocos ejemplos. Debido a que nuestros ejemplos contienen ejemplos de "negativos", alentamos al modelo a comportarse correctamente en este caso

In [14]:
structured_llm.invoke(messages + [message_no_extraction])

Data(people=[])

Ver esta [guía](https://python.langchain.com/docs/how_to/extraction_examples/) se puede obtener más detalles sobre los flujos de trabajo de extracción con ejemplos de referencia, incluida la forma de incorporar plantillas de solicitud y personalizar la generación de mensajes de ejemplo